### Validation
Validating a credit card involves the Luhn algorithm works as follows:
1. The Luhn algorithm starts from the last digit which is called the check digit. Then moving left from this check digit (←), double the value of every digit at even indices.
2. If the result of this doubling operation is greater than 9 (e.g., 6 × 2 = 12), then subtract 9 from the result (e.g., 12: 12 − 9 = 3) or, equivalently, add the digits of the result (e.g., 12: 1 + 2 =3).
3. Now sum all the digits (including the check digit).
4. If the total is divisible by 10 then the number is valid; otherwise, it is not valid.

In [ ]:
# Validate the card the old fashioned way, function accepts 1 arguement: number, returns true/false plus issuer

def validate(number: str):
    card_number_lst = [int(num) for num in number]
    check_digit = card_number_lst.pop(-1)
    card_number_lst.reverse()
    card_number_lst = [num * 2 if i % 2 == 0 else num for i, num in enumerate(card_number_lst)]
    card_number_lst = [num - 9 if num > 9 else num for num in card_number_lst]
    total = sum(card_number_lst) + check_digit
    return total % 10 == 0
validate('345106858850943')

### Card Type
Only looking to determine if the number belongs to these 4 issuers: Visa, MasterCard, American Express, Discover
A card from a given issuer always starts with this number:
- Visa: 4
- MasterCard: 51, 52, 53, 54, 55
- American Express: 34, 37
- Discover: 6011

In [ ]:
def card_type(number: str):
    if number[0] == '4':
        return 'Visa'
    elif number[:2] in ['51', '52', '53', '54', '55']:
        return 'MasterCard'
    elif number[:2] in ['34', '37']:
        return 'American Express'
    elif number[:4] == '6011':
        return 'Discover'
    else:
        return 'Unknown'

card_type('345106858850943')

### Generation
To make this efficient and mitigate the need to post my own CC numbers on the internet, I have written the following function that generates a valid credit card number for a given issuer.

In [ ]:
import random
visa = ['4']
mastercard = ['51', '52', '53', '54', '55']
discover = ['6011']
amex = ['34', '37']
def generate(issuer: str) -> str:
    if issuer == 'visa':
        return generate_credit_card(visa[0], 16)
    elif issuer == 'mastercard':
        return generate_credit_card(random.choice(mastercard),16)
    elif issuer == 'discover':
        return generate_credit_card(discover[0],16)
    elif issuer == 'amex':
        return generate_credit_card(random.choice(amex),15)
    else:
        return 'Unknown'


def generate_credit_card(prefix: str = '', length: int = 16) -> str:
    if length < 2:
        raise ValueError("Length must be at least 2")
    if len(prefix) > 4:
        raise ValueError("Prefix length must be at most 4")

    # Generate the random part of the credit card number
    random_part_length = length - len(prefix)
    random_part = ''.join(str(random.randint(0, 9)) for _ in range(random_part_length - 1))

    # Combine prefix and random part
    card_number = prefix + random_part
    digits = [int(digit) for digit in card_number]
    odd_digits = digits[-1::-2]
    even_digits = digits[-2::-2]
    checksum = sum(even_digits) + sum(sum(divmod(2 * digit, 10)) for digit in odd_digits)
    check_digit = 9 * checksum % 10

    # Append the check digit to the credit card number
    card_number += str(check_digit)

    return card_number    
    
[generate('visa'),
generate('mastercard'),
generate('discover'),
generate('amex')]

In [ ]:
# Put it all together
numbers = [generate('visa'),
 generate('mastercard'),
 generate('discover'),
 generate('amex')]
for number in numbers:
    print(f'{number} is a {card_type(number)} card and is {validate(number)}')

### Editorial
This is very easy code to write (I swiped all of it from the web), and it executes very quickly. It is obviously more efficient and simpler than using an LLM for the purpose of validating credit card numbers, however, it requires that someone know python (or coding in general). The directions for verifying a credit card number have been expressed in plain english above, and a person could be given those instructions and verify credit card numbers, no python necessary. A person with no technical expertise could also give these guidelines to an LLM and ask the LLM to verify credit card numbers. There are products coming onto the market that abstract away all the implementation and deployment steps of setting up an LLM to do something, and allow you to create processes out of LLM blocks, passing the output of one in as the input to the next. Rather than each block executing computer code, each block queries an LLM with instructions that can be given in human language. This empowers the non-technical to build functions that can automate tasks. We are going to explore whether an LLM can capably validate CC numbers given instructions in english. We will also see if an LLM can generate a valid CC number given instructions in english.